# Welcome to this incredible analysis


Let's start by loading the data and importing packages.

In [1]:
import pandas as pd 
import numpy as np 
from scipy import stats

In [2]:
csv = 'data/brainsize.csv'
data = pd.read_csv(csv, sep=';', index_col=0, na_values=".") # Missing values are ID'd as such.
data.head()

,Gender,FSIQ,VIQ,PIQ,Weight,Height,MRI_Count
1,Female,133,132,124,118.0,64.5,816932
2,Male,140,150,124,NaN,72.5,1001121
3,Male,139,123,150,143.0,73.3,1038437
4,Male,133,129,128,172.0,68.8,965353
5,Female,137,132,134,147.0,65.0,951545


Let's generate our variable of interest: partY

In [25]:
# This cell is commented out so it isn't run with the rest. This cell generated the partY data.

#### What is the number of rows?
# rows = data.shape[0]
#### Generate the data
# partY = np.random.rand(rows, 1)
#### Save the data to .txt file
# partY = partY.tolist()
# with open('data/partY.txt', 'w') as fhandle:
#     for x in partY:
#         fhandle.write('%s\n' % x)

Now that the variable of interest is saved to file, let's add it to the dataframe!

In [33]:
partY = []
with open('data/partY.txt', 'r') as fhandle:
    for line in fhandle:
        line = line.replace("]\n", "") # Remove superfluous characters
        line = line.replace("[", "")
        partY.append(line)
data['partY'] = partY
data_desc = data.describe() # For easy VS Code checking